## Deliverable 2. Create a Customer Travel Destinations Map.

In [16]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import os
# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [18]:
# 1. Import the WeatherPy_database.csv file.

city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,City Country,Current Description
0,0,Capoterra,39.1763,8.9718,76.53,77,0,2.30,IT,clear sky
1,1,Ushuaia,-54.8000,-68.3000,33.46,80,40,8.05,AR,scattered clouds
2,2,Mataura,-46.1927,168.8643,47.48,90,41,5.12,NZ,scattered clouds
3,3,Qaanaaq,77.4840,-69.3632,38.91,97,100,6.31,GL,light rain
4,4,Nanortalik,60.1432,-45.2371,45.75,75,100,4.50,GL,overcast clouds


In [19]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 

min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 80
What is the maximum temperature you would like for your trip? 95


In [20]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.

preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,City Country,Current Description
17,17,Road Town,18.4167,-64.6167,82.44,69,40,9.22,VG,scattered clouds
20,20,Kailua,21.4022,-157.7394,84.18,84,100,11.50,US,overcast clouds
36,36,Dogondoutchi,13.6393,4.0287,84.56,55,98,12.95,NE,overcast clouds
37,37,Sibolga,1.7427,98.7792,81.19,81,100,0.98,ID,overcast clouds
40,40,Hithadhoo,-0.6000,73.0833,83.17,73,20,13.38,MV,few clouds
42,42,Kloulklubed,7.0419,134.2556,84.70,72,36,5.84,PW,scattered clouds
44,44,Gaoyou,32.7858,119.4278,89.01,69,100,4.56,CN,overcast clouds
50,50,La Ronge,55.1001,-105.2842,86.22,32,75,16.11,CA,broken clouds
51,51,Georgetown,5.4112,100.3354,83.89,91,20,2.30,MY,few clouds
52,52,Butaritari,3.0707,172.7902,82.18,78,5,18.10,KI,light rain


In [21]:
# 4a. Determine if there are any empty rows.

preferred_cities_df.count()

City_ID                84
City                   84
Lat                    84
Lng                    84
Max Temp               84
Humidity               84
Cloudiness             84
Wind Speed             84
City Country           84
Current Description    84
dtype: int64

In [54]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.

clean_df = preferred_cities_df[["City", "Max Temp", "Lat", "Lng", "Current Description", "City Country"]].copy()
clean_df["Hotel Name"] = ""
clean_df.head(10)


,City,Max Temp,Lat,Lng,Current Description,City Country,Hotel Name
17,Road Town,82.44,18.4167,-64.6167,scattered clouds,VG,
20,Kailua,84.18,21.4022,-157.7394,overcast clouds,US,
36,Dogondoutchi,84.56,13.6393,4.0287,overcast clouds,NE,
37,Sibolga,81.19,1.7427,98.7792,overcast clouds,ID,
40,Hithadhoo,83.17,-0.6000,73.0833,few clouds,MV,
42,Kloulklubed,84.70,7.0419,134.2556,scattered clouds,PW,
44,Gaoyou,89.01,32.7858,119.4278,overcast clouds,CN,
50,La Ronge,86.22,55.1001,-105.2842,broken clouds,CA,
51,Georgetown,83.89,5.4112,100.3354,few clouds,MY,
52,Butaritari,82.18,3.0707,172.7902,light rain,KI,


In [55]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Max Temp", "Current Description", "Lat", "Lng", "City Country"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Max Temp,Current Description,Lat,Lng,City Country,Hotel Name
17,Road Town,82.44,scattered clouds,18.4167,-64.6167,VG,
20,Kailua,84.18,overcast clouds,21.4022,-157.7394,US,
36,Dogondoutchi,84.56,overcast clouds,13.6393,4.0287,NE,
37,Sibolga,81.19,overcast clouds,1.7427,98.7792,ID,
40,Hithadhoo,83.17,few clouds,-0.6000,73.0833,MV,
42,Kloulklubed,84.70,scattered clouds,7.0419,134.2556,PW,
44,Gaoyou,89.01,overcast clouds,32.7858,119.4278,CN,
50,La Ronge,86.22,broken clouds,55.1001,-105.2842,CA,
51,Georgetown,83.89,few clouds,5.4112,100.3354,MY,
52,Butaritari,82.18,light rain,3.0707,172.7902,KI,


In [56]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key,
}


In [57]:
# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"    
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()  
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"] 
    except (IndexError):
        print("Hotel not found... skipping.")

              
        

Hotel not found... skipping.
Hotel not found... skipping.


In [64]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.dropna(subset = ['Hotel Name'])

,City,Max Temp,Current Description,Lat,Lng,City Country,Hotel Name
17,Road Town,82.44,scattered clouds,18.4167,-64.6167,VG,"ZINGARA 76ft Catamaran, full crew-all included"
20,Kailua,84.18,overcast clouds,21.4022,-157.7394,US,Sheffield House Bed & Breakfast
36,Dogondoutchi,84.56,overcast clouds,13.6393,4.0287,NE,Hôtel Alheri Maison Alheri
37,Sibolga,81.19,overcast clouds,1.7427,98.7792,ID,Hotel Syariah CN Darussalam
40,Hithadhoo,83.17,few clouds,-0.6000,73.0833,MV,Scoop Guest House
...,...,...,...,...,...,...,...
393,Huangmei,91.90,overcast clouds,30.1924,116.0250,CN,Tianxiachan Hotel
394,Morgan City,81.01,overcast clouds,29.6994,-91.2068,US,Hampton Inn & Suites Morgan City
395,Gwadar,85.51,scattered clouds,25.1216,62.3254,PK,Sadaf Resort
398,Meulaboh,80.17,few clouds,4.1363,96.1285,ID,Kost Pria


In [65]:
# 8a. Create the output File (CSV)
hotel_df.to_csv('WeatherPy_vacation.csv',index_label='City_ID')
# 8b. Export the City_Data into a csv


In [66]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{City Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [67]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))